In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300 # eval period
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

In [110]:

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [10]:

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)

n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

tensor([[57, 53,  1, 46, 47, 45, 46,  1],
        [26, 53, 58,  1, 57, 53, 10,  0]])

In [55]:

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [74]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [94]:

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [96]:

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.6100, val loss 4.6357
step 300: train loss 2.8261, val loss 2.8259
step 600: train loss 2.5469, val loss 2.5813
step 900: train loss 2.4993, val loss 2.5217
step 1200: train loss 2.4845, val loss 2.5041
step 1500: train loss 2.4696, val loss 2.5039
step 1800: train loss 2.4637, val loss 2.5008
step 2100: train loss 2.4680, val loss 2.4988
step 2400: train loss 2.4538, val loss 2.4917
step 2700: train loss 2.4633, val loss 2.4953


In [109]:

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))




ESalich acand he Pa vaS:
Tyoputherard Lorouganonfrin:
MAlin, heang ryollltonersoro t ay,
Thase Tht,
Bred Yo anspshaieands
bed, mpo t I s,
INTIOfof Whe be sar ar, bean;
Per. w?
H:
CHO: soust'sal.


Yoldou tinor ld ilicry s, my waind our'th gakees howen ndetherselld nds, ay yshe wheros LTHe.
YOuphorughy id by'syoze's icoosthinthes?

Yowing h ceneldathang nighovee, ches d se l se go hakeathe tis scknes b'su ye gensacont s tony are ourt s s rthaknives:
I o th tnker me, se: t aleitl tofren selkelthe


In [135]:
import torch
import torch.nn.functional as F

# Suppose sequence length is 4
T = 3
block_size = 4

# Example attention scores (before masking)
wei = torch.randn(T, T)
print("Original attention scores:\n", wei)

Original attention scores:
 tensor([[-0.3842, -0.1365,  1.7780],
        [-1.6889, -0.4908, -1.2100],
        [ 1.6904, -0.4962,  1.4625]])


In [150]:
x = torch.randint(12, (3,2,4))
x

tensor([[[ 0,  9,  1,  3],
         [ 7,  8,  0, 10]],

        [[10,  3, 10,  3],
         [ 2, 10,  5,  0]],

        [[ 6,  2,  4,  8],
         [ 3,  2,  8, 11]]])

In [151]:
x_t = x.transpose(-1, -2)
x_t


tensor([[[ 0,  7],
         [ 9,  8],
         [ 1,  0],
         [ 3, 10]],

        [[10,  2],
         [ 3, 10],
         [10,  5],
         [ 3,  0]],

        [[ 6,  3],
         [ 2,  2],
         [ 4,  8],
         [ 8, 11]]])

In [118]:

# Lower-triangular mask
tril = torch.tril(torch.ones(block_size, block_size))
print("\nLower-triangular mask (tril):\n", tril)


Lower-triangular mask (tril):
 tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])


In [119]:

# Apply mask: set upper-triangular positions to -inf
wei_masked = wei.masked_fill(tril[:T, :T] == 0, float('-inf'))
print("\nMasked attention scores:\n", wei_masked)


Masked attention scores:
 tensor([[-0.4931,    -inf,    -inf],
        [ 0.2050, -0.7801,    -inf],
        [ 0.0850,  1.0277, -0.3999]])


In [120]:

# Apply softmax
attn_weights = F.softmax(wei_masked, dim=-1)
print("\nSoftmax after masking (attention weights):\n", attn_weights)



Softmax after masking (attention weights):
 tensor([[1.0000, 0.0000, 0.0000],
        [0.7281, 0.2719, 0.0000],
        [0.2391, 0.6137, 0.1472]])
